In [39]:
import pandas as pd
import itertools
import random
import numpy as np
from tqdm import tqdm

In [40]:
df = pd.read_csv('../data/normalized_one_hot.csv')

In [41]:
df

,Pathogenic?,history_class,ethnicity,cancer_dx,known_brca,cancer_dx_type_1,cancer_dx_age_1,cancer_dx_type_2,cancer_dx_age_2,cancer_dx_type_3,...,rel_23_cancer_4,rel_23_cancer_5,rel_23_cancer_6,rel_23_cancer_7,rel_23_cancer_8,rel_23_cancer_9,rel_23_cancer_10,rel_23_age_1,rel_23_age_2,rel_23_age_3
0,0,3,6,1,-1,57,57,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
1,0,3,6,0,-1,32,32,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
2,0,3,2,1,-1,12,37,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
3,0,3,6,1,-1,40,55,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
4,0,3,6,1,-1,55,58,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,0,0,6,0,-1,0,-1,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
2209,0,0,6,0,-1,0,-1,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
2210,0,0,5,1,-1,13,22,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
2211,0,0,6,0,0,0,-1,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1


In [42]:
def gen_df(row, all_rels):
    for x in range(1, 11):
    
        d_init = {
            'Pathogenic?': row['Pathogenic?'],
            'history_class': row['history_class'],
            'ethnicity': row['ethnicity'],
            'cancer_dx': row['cancer_dx'],
            'known_brca': row['known_brca'],
            'cancer_dx_type_1': row['cancer_dx_type_1'],
            'cancer_dx_age_1': row['cancer_dx_age_1'],
            'cancer_dx_type_2': row['cancer_dx_type_2'],
            'cancer_dx_age_2': row['cancer_dx_age_2'],
            'cancer_dx_type_3': row['cancer_dx_type_3'],
            'cancer_dx_age_3': row['cancer_dx_age_3'],
        }

        for y in range(1, 24):
            d_init[f"rel_{y}_relationship"] = all_rels[x-1]["relationship"]

            for z in range(len(all_rels[x-1]["cancers"])):
                d_init[f"rel_{y}_cancer_{z+1}"] = all_rels[x-1]["cancers"][z]
            
            for w in range(len(all_rels[x-1]["ages"])):
                d_init[f"rel_{y}_age_{w+1}"] = all_rels[x-1]["ages"][w]
        
        for k, v in d_init.items():
            d_init[k] = [int(v)]
        new_df = pd.DataFrame(data = d_init)
        return new_df

In [43]:
# new_df = pd.DataFrame()

for index, row in tqdm(df.iterrows()):
    all_rel_data = []
    for i in range(1, 24):
        # rel relation 1-23
        # rel cancer 1-10
        # rel age 1-3
        #gets each rel columns
        rel_cols = [x for x, y in row.items() if f"rel_{i}_" in x]
        
        #specific to each relative
        rel_relations = [row[name] for name in rel_cols if 'relationship' in name]
        rel_cancers = [row[name] for name in rel_cols if 'cancer' in name]
        rel_ages = [row[name] for name in rel_cols if 'age' in name]
        
        all_rel_data.append({
            'relationship': rel_relations[0],
            'cancers': rel_cancers,
            'ages': rel_ages
        })
    
    filled_rel_data = [x for x in all_rel_data if x['relationship'] != 0 and len(np.unique(x['cancers'])) != 1 and len(np.unique(x['ages'])) != 1]
    unfilled_rel_data = [x for x in all_rel_data if x['relationship'] == 0 and len(np.unique(x['cancers'])) == 1 and len(np.unique(x['ages'])) == 1]
#     print(filled_rel_data)
#     print("")
#     print(unfilled_rel_data)
    
    if len(filled_rel_data) > 5: # 5 for augmentation 2
        shuffled_filled_rel_data = []
        for p in range(1, 4):
            shuffled_filled_rel_data.append(random.sample(filled_rel_data, len(filled_rel_data)))
    else:
        shuffled_filled_rel_data = [list(x) for x in list(itertools.permutations(filled_rel_data)) if list(x) != filled_rel_data]

    for perm in shuffled_filled_rel_data:
        combined = perm + unfilled_rel_data
        new_df = gen_df(row, combined)
        df = df.append(new_df)

2213it [07:30,  4.91it/s] 


In [44]:
df

,Pathogenic?,history_class,ethnicity,cancer_dx,known_brca,cancer_dx_type_1,cancer_dx_age_1,cancer_dx_type_2,cancer_dx_age_2,cancer_dx_type_3,...,rel_23_cancer_4,rel_23_cancer_5,rel_23_cancer_6,rel_23_cancer_7,rel_23_cancer_8,rel_23_cancer_9,rel_23_cancer_10,rel_23_age_1,rel_23_age_2,rel_23_age_3
0,0,3,6,1,-1,57,57,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
1,0,3,6,0,-1,32,32,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
2,0,3,2,1,-1,12,37,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
3,0,3,6,1,-1,40,55,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
4,0,3,6,1,-1,55,58,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,0,6,0,-1,0,-1,0,-1,0,...,0,0,0,0,0,0,0,78,-1,-1
0,0,0,6,0,-1,0,-1,0,-1,0,...,0,0,0,0,0,0,0,62,-1,-1
0,0,0,6,0,-1,0,-1,0,-1,0,...,0,0,0,0,0,0,0,62,-1,-1
0,0,0,6,0,-1,0,-1,0,-1,0,...,0,0,0,0,0,0,0,76,-1,-1


In [45]:
df = df.astype(int)

In [46]:
df.to_csv('../data/reaugmented_2.csv', index = False)